# BLEU

https://github.com/moses-smt/mosesdecoder/blob/master/scripts/generic/multi-bleu.perl

## Results

From: <a href="../paper/webnlg2017/The WebNLG Challenge -  Generating Text from RDF Data.pdf">The WebNLG Challenge - Generating Text from RDF Data</a>

<img src='../img/bleu_result.jpg' />

## Melbourne

<a href="../data/webnlg2017/submissions/melbourne/WebNLG Challenge - Bayu Distiawan T.pdf">report</a>

In [1]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/GKB_Unimelb_all-cat.txt

BLEU = 45.13, 79.9/55.4/38.8/27.4 (BP=0.969, ratio=0.969, hyp_len=40642, ref_len=41921)


In [3]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/GKB_Unimelb_all-cat.txt | tail -10

castle is english language .
eric flint was born in burbank , california .
macmillan publishers is the parent company of farrar , straus and giroux .
john cowper powys was a glastonbury romance .
soho press is in united states .
the secret scripture was published by faber and faber .
asian americans are an ethnic group in united states .
english language is spoken in united states .
weymouth sands was preceded by a glastonbury romance .
the manager of a . c . chievo verona is rolando maran .


## Tilb-SMT

<a href="../data/webnlg2017/submissions/tilburg/report.pdf">report</a>

In [16]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-2_all-cat.txt | tail -10

novel castle is english language
. eric flint was born in burbank , california .
macmillan publishers is the parent company of farrar , straus and giroux
john cowper powys notablework a glastonbury romance
soho press , united states .
, published by faber and faber
asian americans are an ethnic group .
english is the language of the united states .
weymouth sands was preceded by a glastonbury romance
the manager of a . c . chievo verona is rolando maran


In [15]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-2_all-cat.txt

BLEU = 44.28, 74.4/53.0/37.5/26.8 (BP=0.992, ratio=0.992, hyp_len=42193, ref_len=42521)


## PKUWriter

<a href="../data/webnlg2017/submissions/pkuwriter/PKUWriter_system.pdf">report</a>

In [2]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/PKUWriter_all-cat.txt | tail -10

the novel into battle is written by the english language .
john cowper powys was born in burbank , california .
the parent company of the macmillan publishers press is the macmillan publishers company of the macmillan publishers .
the novel owen glendower is a notable work by john cowper powys .
soho press is located in united states .
the polish academy for writtenwork is referencenumber _ in _ the _ national _ register _ of _ historic _ places .
asian americans are an ethnic group in the united states .
english is the language spoken in the united states .
the novel into battle is preceded by a glastonbury romance .
rolando maran manages the a . c . chievo verona .


In [5]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/PKUWriter_all-cat.txt

BLEU = 39.88, 73.9/49.9/35.6/26.0 (BP=0.927, ratio=0.930, hyp_len=38248, ref_len=41143)


## UPF-FORGe

<a href="../data/webnlg2017/submissions/upf/WebNLG_V0.2.pdf">report</a>

In [4]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UPF-TALN_all-cat.txt | tail -10

the english language is spoken in castle ( novel ) .
eric flint was born in burbank ( california ) .
the parent company of farrar , straus and giroux is macmillan publishers .
a glastonbury romance wrote john cowper powys .
soho press is published in the united states .
the publisher of the secret scripture is faber and faber .
a ethnic group of the united states are asian americans .
the english language is spoken in the united states .
weymouth sands follows a glastonbury romance .
rolando maran is the manager of a . c . chievo verona .


In [6]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UPF-TALN_all-cat.txt

BLEU = 38.65, 75.5/48.7/31.1/19.5 (BP=1.000, ratio=1.064, hyp_len=47234, ref_len=44405)


## Tilb-Pipeline

<a href="../data/webnlg2017/submissions/tilburg/report.pdf">report</a>

In [5]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-3_all-cat.txt | tail -10

castle is in english .
eric flint was born in burbank , california .
macmillan publishers owns farrar , straus , and giroux .
a glastonbury romance is a notable work by john cowper powys .
soho press is in united states .
the secret scripture publisher of faber and faber .
asian americans live in united states .
united states is in english .
weymouth sands was preceded by a glastonbury romance .
rolando maran manages a . c . chievo verona .


In [8]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-3_all-cat.txt

BLEU = 35.29, 81.8/56.6/38.8/26.5 (BP=0.755, ratio=0.781, hyp_len=30335, ref_len=38854)


## Tilb-NMT

<a href="../data/webnlg2017/submissions/tilburg/report.pdf">report</a>

In [6]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-1_all-cat.txt | tail -10

english is the language of castle .
eric flint was born in burbank , california .
macmillan publishers is the parent company of farrar , straus , and giroux .
a glastonbury romance is a notable work by john cowper powys .
soho press is located in united states .
th secret scriptures was published by faber and faber .
asian americans are an ethnic group in united states .
english is the language of united states .
weymouth sands was preceded by a glastonbury romance .
the manager of a . c . chievo verona is rolando maran .


In [9]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-1_all-cat.txt

BLEU = 34.60, 68.8/42.7/27.5/17.7 (BP=1.000, ratio=1.054, hyp_len=45605, ref_len=43278)


## Baseline

In [7]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Baseline_all-cat.txt | tail -10

the english language is spoken in the republic of ireland .
author was born in burbank , california .
the parent company of wiley - blackwell is macmillan publishers .
a glastonbury romance was written by author .
lahore is in the united states .
the parent company of wiley - blackwell is parentcompany .
asian americans are an ethnic group in the united states .
english is the language of the united states .
a glastonbury romance was preceded by precededby .
the manager of a . c . chievo verona is rolando maran .


In [10]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Baseline_all-cat.txt

BLEU = 33.24, 65.4/42.3/31.8/24.8 (BP=0.865, ratio=0.873, hyp_len=35312, ref_len=40435)


## Adapt

<a href="../data/webnlg2017/submissions/adaptCenter/WebNLG_system.pdf">report</a>

In [8]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/ADAPT_Centre_all-cat.txt | tail -10

the novel castle is the language spoken in the english language .
eric flint was born in burbank , california .
macmillan publishers is the parent company of farrar , straus and giroux .
a glastonbury romance is the notable work of john cowper powys .
soho press is located in the united states .
the novel the secret scriptures , is published by faber and faber .
asian americans are an ethnic group in the u . s .
english is the language of the united states .
a glastonbury romance is the sequel to weymouth sands .
the manager of a . c . chievo verona is rolando maran .


In [11]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/ADAPT_Centre_all-cat.txt

BLEU = 31.06, 54.9/35.7/25.3/18.7 (BP=1.000, ratio=1.337, hyp_len=59727, ref_len=44663)


## UIT-VNU

<a href="../data/webnlg2017/submissions/uit-danglt-clnlp/Phong-Nguyen-Tran_Dang-Tuan-Nguyen.docx">report</a>

In [9]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UIT-DANGNT-CLNLP_all-cat.txt | tail -10

english language is one of the languages that is spoken in castle ( novel ) .
the birth place of eric flint is burbank , california .
macmillan publishers is the parent company of the farrar , straus and giroux .
null
soho press is located within the country of the united states .
the book the secret scripture was published by faber and faber .
asian americans are part of one of the ethnic groups in the united states .
english language is one of the languages that is spoken in united states .
a glastonbury romance was written prior to weymouth sands .
rolando maran is the manager of the a . c . chievo verona .


In [12]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UIT-DANGNT-CLNLP_all-cat.txt

BLEU = 7.07, 67.5/54.4/39.5/29.2 (BP=0.156, ratio=0.350, hyp_len=11219, ref_len=32072)


# METEOR

http://www.cs.cmu.edu/~alavie/METEOR/

## Results

From: <a href="../paper/webnlg2017/The WebNLG Challenge -  Generating Text from RDF Data.pdf">The WebNLG Challenge - Generating Text from RDF Data</a>

<img src='../img/meteor_result.jpg' />

## UPF-FORGe

<a href="../data/webnlg2017/submissions/upf/WebNLG_V0.2.pdf">report</a>

In [14]:
!java -Xmx2G -jar ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UPF-TALN_all-cat.txt ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.meteor -l en -norm -r 3 -a ../evaluation/webnlg2017/meteor-1.5/data/paraphrase-en.gz

Meteor version: 1.5

Eval ID:        meteor-1.5-wo-en-norm-0.85_0.2_0.6_0.75-ex_st_sy_pa-1.0_0.6_0.8_0.6

Language:       English
Format:         plaintext
Task:           Ranking
Modules:        exact stem synonym paraphrase
Weights:        1.0 0.6 0.8 0.6
Parameters:     0.85 0.2 0.6 0.75

Segment 1 score:	0.3697195430486158
Segment 2 score:	0.5659428112032991
Segment 3 score:	0.5603293785486797
Segment 4 score:	0.4937280319092536
Segment 5 score:	0.5887851481584605
Segment 6 score:	0.44612525909102846
Segment 7 score:	0.46837277027616664
Segment 8 score:	0.5309315227410509
Segment 9 score:	0.521324221008186
Segment 10 score:	0.9798850574712644
Segment 11 score:	0.45671713505159706
Segment 12 score:	0.5472108090547545
Segment 13 score:	0.4340618097174766
Segment 14 score:	0.39070927010803513
Segment 15 score:	0.39857703619888607
Segment 16 score:	0.5338212580268122
Segment 17 score:	0.5489238212448153
Segment 18 score:	0.3546503291117034
Segment 19 score:	0.4288044751134423
Segment 2

Segment 252 score:	0.46546042329099685
Segment 253 score:	0.44605636094732304
Segment 254 score:	0.3891285158401767
Segment 255 score:	0.37635782289782543
Segment 256 score:	0.41177089227117447
Segment 257 score:	0.3455470955224817
Segment 258 score:	0.4580921014321232
Segment 259 score:	0.33636612093830665
Segment 260 score:	0.5053927878976566
Segment 261 score:	0.36760872833459235
Segment 262 score:	0.287069894741754
Segment 263 score:	0.47258035937396314
Segment 264 score:	0.2957480769724148
Segment 265 score:	0.5971798599287671
Segment 266 score:	0.30919930579074745
Segment 267 score:	0.11451942740286299
Segment 268 score:	0.3195705821578627
Segment 269 score:	0.3780848208233248
Segment 270 score:	0.3107368874190316
Segment 271 score:	0.30420172672267504
Segment 272 score:	0.4023863924091467
Segment 273 score:	0.40483692095772666
Segment 274 score:	0.5363929295120985
Segment 275 score:	0.27258250208943513
Segment 276 score:	0.4172965919868134
Segment 277 score:	0.24390717842265247


Segment 475 score:	0.4432989558992796
Segment 476 score:	0.4336452752037022
Segment 477 score:	0.4069237158557947
Segment 478 score:	0.4032696469443561
Segment 479 score:	0.43853065200444813
Segment 480 score:	0.4449740167419749
Segment 481 score:	0.4031669036396115
Segment 482 score:	0.43728184602075165
Segment 483 score:	0.45933035900750935
Segment 484 score:	0.33293949665438916
Segment 485 score:	0.4017759747745401
Segment 486 score:	0.3819599538075271
Segment 487 score:	0.43283725453740635
Segment 488 score:	0.4118367501203072
Segment 489 score:	0.40273200379097013
Segment 490 score:	0.4086052604585694
Segment 491 score:	0.41164362750018935
Segment 492 score:	0.4386977058653837
Segment 493 score:	0.41808054704337405
Segment 494 score:	0.3839190640465339
Segment 495 score:	0.6070481374254909
Segment 496 score:	0.3990817161708573
Segment 497 score:	0.4679594318243194
Segment 498 score:	0.423656065572424
Segment 499 score:	0.47615207372559915
Segment 500 score:	0.367905452490724
Segme

Segment 694 score:	0.43466609010215546
Segment 695 score:	0.43636322062867067
Segment 696 score:	0.4052176998879933
Segment 697 score:	0.41433472746975575
Segment 698 score:	0.3468548968664829
Segment 699 score:	0.3307736082456276
Segment 700 score:	0.307214339231524
Segment 701 score:	0.3614510386128334
Segment 702 score:	0.3237307054404582
Segment 703 score:	0.3076916423112249
Segment 704 score:	0.43253076652873607
Segment 705 score:	0.40955669604865885
Segment 706 score:	0.4547533762421891
Segment 707 score:	0.4300236767556123
Segment 708 score:	0.4143395987209832
Segment 709 score:	0.4139537887645307
Segment 710 score:	0.4061685099456724
Segment 711 score:	0.4582762467802907
Segment 712 score:	0.4322188900400689
Segment 713 score:	0.5171544913302593
Segment 714 score:	0.47767781178412005
Segment 715 score:	0.4099709343010623
Segment 716 score:	0.4758709528545699
Segment 717 score:	0.40854214446188136
Segment 718 score:	0.44081827676319274
Segment 719 score:	0.4022842854405031
Segme

Segment 928 score:	0.3673679040063982
Segment 929 score:	0.43633561036540475
Segment 930 score:	0.4262133555131258
Segment 931 score:	0.38009296683263877
Segment 932 score:	0.46854795391338266
Segment 933 score:	0.47356144299822206
Segment 934 score:	0.3856696734318783
Segment 935 score:	0.3759692423687571
Segment 936 score:	0.3538437856552692
Segment 937 score:	0.36158592649892246
Segment 938 score:	0.40343068508706564
Segment 939 score:	0.34829315558326707
Segment 940 score:	0.4297533268633011
Segment 941 score:	0.3747991096952841
Segment 942 score:	0.37277403728919745
Segment 943 score:	0.3895444622607869
Segment 944 score:	0.45689003571731673
Segment 945 score:	0.463519484981654
Segment 946 score:	0.3956493989112333
Segment 947 score:	0.3877517424036706
Segment 948 score:	0.44524257420707186
Segment 949 score:	0.39808815383265045
Segment 950 score:	0.4073101247230196
Segment 951 score:	0.38235265352020487
Segment 952 score:	0.33253104647914944
Segment 953 score:	0.40506376311476744

Segment 1158 score:	0.31368369408248653
Segment 1159 score:	0.36448655362831317
Segment 1160 score:	0.47643551543639595
Segment 1161 score:	0.26501364489879387
Segment 1162 score:	0.36317858433577194
Segment 1163 score:	1.0
Segment 1164 score:	1.0
Segment 1165 score:	0.389486873058784
Segment 1166 score:	0.33445420019002603
Segment 1167 score:	0.35294331746589697
Segment 1168 score:	1.0
Segment 1169 score:	0.42750933026297866
Segment 1170 score:	0.3546503291117034
Segment 1171 score:	0.26670761379759006
Segment 1172 score:	0.27614490014207577
Segment 1173 score:	0.39293359075533385
Segment 1174 score:	0.5502342065112328
Segment 1175 score:	0.5613051214200641
Segment 1176 score:	0.5329969227082326
Segment 1177 score:	0.42843823789397756
Segment 1178 score:	0.3766372578511156
Segment 1179 score:	0.31610551335350767
Segment 1180 score:	0.5299621331714771
Segment 1181 score:	1.0
Segment 1182 score:	0.5501286920112574
Segment 1183 score:	0.36960969705562297
Segment 1184 score:	0.25687277053

Segment 1400 score:	0.3419782388858276
Segment 1401 score:	0.30227070403046324
Segment 1402 score:	0.3904030891528622
Segment 1403 score:	0.4905512521468981
Segment 1404 score:	0.4185275299685884
Segment 1405 score:	0.5225381093655163
Segment 1406 score:	0.4838397805141846
Segment 1407 score:	0.42642657609707385
Segment 1408 score:	0.3941015332453851
Segment 1409 score:	0.4527774973335174
Segment 1410 score:	0.2872064901997175
Segment 1411 score:	0.31331511180900584
Segment 1412 score:	0.34676464174404237
Segment 1413 score:	0.3143014071477373
Segment 1414 score:	0.38016052782945153
Segment 1415 score:	0.39687574522821023
Segment 1416 score:	0.4202136084854365
Segment 1417 score:	0.39720564764901717
Segment 1418 score:	0.45342521001936686
Segment 1419 score:	0.4080266645842542
Segment 1420 score:	0.4960583754908626
Segment 1421 score:	0.28616865825766424
Segment 1422 score:	0.3822619424237608
Segment 1423 score:	0.4357619859345253
Segment 1424 score:	0.35064214476044453
Segment 1425 sc

Segment 1612 score:	0.3086345401878094
Segment 1613 score:	0.2810786824219854
Segment 1614 score:	0.2827059676201065
Segment 1615 score:	0.31991002327902834
Segment 1616 score:	0.41535290213693016
Segment 1617 score:	0.29974480598766884
Segment 1618 score:	0.2371096710950052
Segment 1619 score:	0.43184623337331585
Segment 1620 score:	0.433270547865505
Segment 1621 score:	0.3708416560857497
Segment 1622 score:	0.3965374829974642
Segment 1623 score:	0.4158659213455533
Segment 1624 score:	0.418928359311277
Segment 1625 score:	0.3617607932753041
Segment 1626 score:	0.43302775383845327
Segment 1627 score:	0.36020268835745906
Segment 1628 score:	0.35172929893509547
Segment 1629 score:	0.3213323317575088
Segment 1630 score:	0.36208902084530853
Segment 1631 score:	0.35589411656809405
Segment 1632 score:	0.3555578646288392
Segment 1633 score:	0.39563046615882863
Segment 1634 score:	0.38490101599335835
Segment 1635 score:	0.34052671812360763
Segment 1636 score:	0.3576815675890115
Segment 1637 sc

Segment 1826 score:	0.4013158077616238
Segment 1827 score:	0.39857703219594315
Segment 1828 score:	0.48233132896972924
Segment 1829 score:	0.33195461476236454
Segment 1830 score:	0.29603075804427675
Segment 1831 score:	0.397412868963004
Segment 1832 score:	0.24319619227379363
Segment 1833 score:	0.4494009154920323
Segment 1834 score:	0.352955321206155
Segment 1835 score:	0.35505561889243104
Segment 1836 score:	0.4504572047148757
Segment 1837 score:	0.43869787935628923
Segment 1838 score:	0.49014776630674706
Segment 1839 score:	0.34140283915335645
Segment 1840 score:	0.3094412228814388
Segment 1841 score:	0.30463077682376066
Segment 1842 score:	0.29369346315110434
Segment 1843 score:	0.4403542790559091
Segment 1844 score:	0.3027667758834874
Segment 1845 score:	0.31823178352341325
Segment 1846 score:	0.38417517777807025
Segment 1847 score:	0.38203205242187743
Segment 1848 score:	0.3990967341011492
Segment 1849 score:	0.3495662988815135
Segment 1850 score:	0.34634951073660103
Segment 1851

## Tilb-SMT

<a href="../data/webnlg2017/submissions/tilburg/report.pdf">report</a>

In [17]:
!java -Xmx2G -jar ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-2_all-cat.txt ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.meteor -l en -norm -r 3 -a ../evaluation/webnlg2017/meteor-1.5/data/paraphrase-en.gz

Meteor version: 1.5

Eval ID:        meteor-1.5-wo-en-norm-0.85_0.2_0.6_0.75-ex_st_sy_pa-1.0_0.6_0.8_0.6

Language:       English
Format:         plaintext
Task:           Ranking
Modules:        exact stem synonym paraphrase
Weights:        1.0 0.6 0.8 0.6
Parameters:     0.85 0.2 0.6 0.75

Segment 1 score:	0.4982660839324799
Segment 2 score:	0.6171450358153345
Segment 3 score:	0.3329984916316827
Segment 4 score:	0.26972694527454394
Segment 5 score:	0.26835374270917167
Segment 6 score:	0.5278948939051884
Segment 7 score:	0.5474647947689384
Segment 8 score:	0.3696020716118977
Segment 9 score:	0.3600668815277863
Segment 10 score:	0.39250293900452154
Segment 11 score:	0.5071638647868559
Segment 12 score:	0.4283926140031914
Segment 13 score:	0.4067890598110925
Segment 14 score:	0.3460022968291308
Segment 15 score:	0.4262282713535066
Segment 16 score:	0.6132617093254663
Segment 17 score:	0.5314258059420802
Segment 18 score:	0.5895027648186086
Segment 19 score:	0.48834340272344423
Segment 2

Segment 246 score:	0.38306317235944
Segment 247 score:	0.4520033193741844
Segment 248 score:	0.3431955049226089
Segment 249 score:	0.522494064319976
Segment 250 score:	0.30388666229960676
Segment 251 score:	0.2764404291792448
Segment 252 score:	1.0
Segment 253 score:	0.4577023008637297
Segment 254 score:	0.31961779145105057
Segment 255 score:	0.5591961832282727
Segment 256 score:	0.3558833387409599
Segment 257 score:	0.32489836589888244
Segment 258 score:	0.24443790004462243
Segment 259 score:	0.14803715438167836
Segment 260 score:	0.493036643337559
Segment 261 score:	0.3677151253358676
Segment 262 score:	0.3412261196181774
Segment 263 score:	0.34313938495578916
Segment 264 score:	0.35455892438770553
Segment 265 score:	0.4537613645224139
Segment 266 score:	0.5207790980042178
Segment 267 score:	0.36438917961689116
Segment 268 score:	0.2847714760866699
Segment 269 score:	0.40020919636468294
Segment 270 score:	0.3673881097850588
Segment 271 score:	0.42888702983394555
Segment 272 score:	0.

Segment 486 score:	0.3017165795476324
Segment 487 score:	0.48735819844390277
Segment 488 score:	0.47189564942700807
Segment 489 score:	0.42183232759425415
Segment 490 score:	0.40120963006350235
Segment 491 score:	0.509128867962933
Segment 492 score:	0.4532931790120768
Segment 493 score:	0.41525602128790146
Segment 494 score:	0.454024226253904
Segment 495 score:	0.6191108093878556
Segment 496 score:	0.40068868022614224
Segment 497 score:	0.4238324496298178
Segment 498 score:	0.44207782383044975
Segment 499 score:	0.46787392854790355
Segment 500 score:	0.40565870216433214
Segment 501 score:	0.4873820925444625
Segment 502 score:	0.3978316343144639
Segment 503 score:	0.45814699862690333
Segment 504 score:	0.41318608899271525
Segment 505 score:	0.3953237895048847
Segment 506 score:	0.427861953623985
Segment 507 score:	0.38450155239582495
Segment 508 score:	0.4835185278279983
Segment 509 score:	0.4603330556808984
Segment 510 score:	0.42771801603143417
Segment 511 score:	0.3769641904063353
Se

Segment 715 score:	0.46099218161015465
Segment 716 score:	0.4684528406438297
Segment 717 score:	0.5569401518529413
Segment 718 score:	0.4460794546947384
Segment 719 score:	0.24737431353671976
Segment 720 score:	0.4122306462459602
Segment 721 score:	0.38278527725085165
Segment 722 score:	0.4228061761040228
Segment 723 score:	0.44771056742918713
Segment 724 score:	0.4158673013764583
Segment 725 score:	0.45419982538285664
Segment 726 score:	0.3820995735049484
Segment 727 score:	0.49324973495593655
Segment 728 score:	0.5273820311427666
Segment 729 score:	0.375131752389127
Segment 730 score:	0.39965306949776275
Segment 731 score:	0.38180805580094823
Segment 732 score:	0.3694473272334019
Segment 733 score:	0.43096072627195836
Segment 734 score:	0.3839127916848092
Segment 735 score:	0.4782960899808724
Segment 736 score:	0.42534848790954344
Segment 737 score:	0.4186683579309031
Segment 738 score:	0.3857408805039567
Segment 739 score:	0.4373985291665614
Segment 740 score:	0.4940294722961545
Seg

Segment 929 score:	0.4488061979287102
Segment 930 score:	0.40652080425061426
Segment 931 score:	0.3768233613149
Segment 932 score:	0.4556168643189319
Segment 933 score:	0.49497634065286084
Segment 934 score:	0.3440850811856208
Segment 935 score:	0.33472514222923755
Segment 936 score:	0.38192354986793375
Segment 937 score:	0.3954027932438827
Segment 938 score:	0.3747521972854192
Segment 939 score:	0.358318733850209
Segment 940 score:	0.39629303066333693
Segment 941 score:	0.40210988439791207
Segment 942 score:	0.40218998284080393
Segment 943 score:	0.3984797859087924
Segment 944 score:	0.4132307277433761
Segment 945 score:	0.44760022153475204
Segment 946 score:	0.43061065517880504
Segment 947 score:	0.4423574464326326
Segment 948 score:	0.4254679381511254
Segment 949 score:	0.42298371296709375
Segment 950 score:	0.4384242869597845
Segment 951 score:	0.3679577489318084
Segment 952 score:	0.34078382210282454
Segment 953 score:	0.3445292728359239
Segment 954 score:	0.41060504965458205
Segm

Segment 1167 score:	0.33536520527141184
Segment 1168 score:	0.5474647947689384
Segment 1169 score:	1.0
Segment 1170 score:	0.35079538006463284
Segment 1171 score:	0.24124730405167952
Segment 1172 score:	0.2610790440258877
Segment 1173 score:	0.39293359075533385
Segment 1174 score:	0.3079337007587296
Segment 1175 score:	0.5139872900426984
Segment 1176 score:	0.36419779372333394
Segment 1177 score:	0.4336185492206869
Segment 1178 score:	0.4336185492206869
Segment 1179 score:	0.43400864379155385
Segment 1180 score:	0.36419779372333394
Segment 1181 score:	0.3292553128676817
Segment 1182 score:	0.2535675286790715
Segment 1183 score:	0.44612525909102846
Segment 1184 score:	0.4332260266195148
Segment 1185 score:	0.38797851314973636
Segment 1186 score:	0.21436853318831597
Segment 1187 score:	0.2964047827911339
Segment 1188 score:	0.38797851314973636
Segment 1189 score:	0.2964047827911339
Segment 1190 score:	0.40342087627277695
Segment 1191 score:	0.40082207415390914
Segment 1192 score:	0.30268

Segment 1387 score:	0.35655103181246783
Segment 1388 score:	0.32960061904995147
Segment 1389 score:	0.40870730738915717
Segment 1390 score:	0.33374299529944146
Segment 1391 score:	0.39754007413515735
Segment 1392 score:	0.3235212850350412
Segment 1393 score:	0.32164574065986073
Segment 1394 score:	0.3906652696486886
Segment 1395 score:	0.36447362622781065
Segment 1396 score:	0.2273911781478727
Segment 1397 score:	0.3169451101574094
Segment 1398 score:	0.2947312077909447
Segment 1399 score:	0.43597756314934594
Segment 1400 score:	0.24590190343643586
Segment 1401 score:	0.19377600334366882
Segment 1402 score:	0.23045466150199917
Segment 1403 score:	0.3987278834721049
Segment 1404 score:	0.4818435853122792
Segment 1405 score:	0.49288528353182015
Segment 1406 score:	0.43776430535266475
Segment 1407 score:	0.38124636635082726
Segment 1408 score:	0.3589101869424097
Segment 1409 score:	0.4383367562886696
Segment 1410 score:	0.3975317432881319
Segment 1411 score:	0.43463635193081723
Segment 14

Segment 1623 score:	0.3326245466125671
Segment 1624 score:	0.3632350715617602
Segment 1625 score:	0.3754864964778289
Segment 1626 score:	0.39668874825484995
Segment 1627 score:	0.3289091654891717
Segment 1628 score:	0.28669283147594987
Segment 1629 score:	0.2931073285730557
Segment 1630 score:	0.3322814628236083
Segment 1631 score:	0.29686336181702894
Segment 1632 score:	0.2907554464483721
Segment 1633 score:	0.28895564941367524
Segment 1634 score:	0.35758735420752535
Segment 1635 score:	0.3331261612055473
Segment 1636 score:	0.29131070370326634
Segment 1637 score:	0.27772669220257185
Segment 1638 score:	0.246763736046767
Segment 1639 score:	0.29104025296531005
Segment 1640 score:	0.25335755605594223
Segment 1641 score:	0.32774121881477847
Segment 1642 score:	0.34058346259070743
Segment 1643 score:	0.3023718063808938
Segment 1644 score:	0.3327922898958663
Segment 1645 score:	0.33639307006939984
Segment 1646 score:	0.33089955971760965
Segment 1647 score:	0.42924624133310063
Segment 1648

Segment 1853 score:	0.34869528057875065
Segment 1854 score:	0.2959946190978695
Segment 1855 score:	0.4373492015775413
Segment 1856 score:	0.3667952686127796
Segment 1857 score:	0.3309112551108941
Segment 1858 score:	0.3344276245395352
Segment 1859 score:	0.28248401272810253
Segment 1860 score:	0.2398720047161359
Segment 1861 score:	0.31060587292924
Segment 1862 score:	0.4091870663549451

System level statistics:


           Test Matches                  Reference Matches
Stage      Content  Function    Total    Content  Function    Total
1            18459     10047    28506      18459     10047    28506
2              101         0      101         99         2      101
3              119        56      175        123        52      175
4              499       548     1047        483       672     1155
Total        19178     10651    29829      19164     10773    29937

Test words:             42193
Reference words:        42317
Chunks:                 11454
Precision:              

## Melbourne

<a href="../data/webnlg2017/submissions/melbourne/WebNLG Challenge - Bayu Distiawan T.pdf">report</a>

In [13]:
!java -Xmx2G -jar ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/GKB_Unimelb_all-cat.txt ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.meteor -l en -norm -r 3 -a ../evaluation/webnlg2017/meteor-1.5/data/paraphrase-en.gz

Meteor version: 1.5

Eval ID:        meteor-1.5-wo-en-norm-0.85_0.2_0.6_0.75-ex_st_sy_pa-1.0_0.6_0.8_0.6

Language:       English
Format:         plaintext
Task:           Ranking
Modules:        exact stem synonym paraphrase
Weights:        1.0 0.6 0.8 0.6
Parameters:     0.85 0.2 0.6 0.75

Segment 1 score:	0.5425269137216473
Segment 2 score:	1.0
Segment 3 score:	0.5280620451812629
Segment 4 score:	0.5183550629438616
Segment 5 score:	1.0
Segment 6 score:	1.0
Segment 7 score:	1.0
Segment 8 score:	1.0
Segment 9 score:	0.5022008728792771
Segment 10 score:	0.9798850574712644
Segment 11 score:	0.3666547512324887
Segment 12 score:	0.5472108090547545
Segment 13 score:	0.5523848588957876
Segment 14 score:	0.5183550629438616
Segment 15 score:	0.6049275687106835
Segment 16 score:	0.4706054944867075
Segment 17 score:	0.5523848588957876
Segment 18 score:	0.534247105415274
Segment 19 score:	1.0
Segment 20 score:	0.46928443699735994
Segment 21 score:	1.0
Segment 22 score:	1.0
Segment 23 score:	0.53

Segment 252 score:	0.5432726247420528
Segment 253 score:	0.46880811410381756
Segment 254 score:	0.32412684455435564
Segment 255 score:	0.5530511702257642
Segment 256 score:	0.36384983247405184
Segment 257 score:	0.46243265251200066
Segment 258 score:	0.5033165144895348
Segment 259 score:	0.3383214107682507
Segment 260 score:	0.47861401242245605
Segment 261 score:	0.38706726349683035
Segment 262 score:	0.3470980796116069
Segment 263 score:	0.5041768807639524
Segment 264 score:	0.290659003899214
Segment 265 score:	0.45734325223056765
Segment 266 score:	0.9733587059942912
Segment 267 score:	0.539377660435147
Segment 268 score:	0.4023029319751272
Segment 269 score:	0.5116629851093241
Segment 270 score:	0.3844423839213888
Segment 271 score:	0.36465342122647554
Segment 272 score:	1.0
Segment 273 score:	0.4505558122710625
Segment 274 score:	0.6021570179606955
Segment 275 score:	0.34413705334003863
Segment 276 score:	0.4994580007010818
Segment 277 score:	0.3412350907309928
Segment 278 score:	0

Segment 484 score:	0.37206422215797275
Segment 485 score:	0.3687382402230882
Segment 486 score:	0.39644450827671796
Segment 487 score:	0.3563356343456942
Segment 488 score:	0.37261244231221147
Segment 489 score:	0.4251596521287827
Segment 490 score:	0.43731317220709737
Segment 491 score:	0.44147015834306536
Segment 492 score:	0.3661710903893752
Segment 493 score:	0.4152379547345818
Segment 494 score:	0.2935715115929534
Segment 495 score:	0.3306596790431243
Segment 496 score:	0.35463498066649796
Segment 497 score:	0.45445631882805587
Segment 498 score:	0.4133244840072544
Segment 499 score:	0.357091490777348
Segment 500 score:	0.3478506870831594
Segment 501 score:	0.4356739586182379
Segment 502 score:	0.36513440072801673
Segment 503 score:	0.419012607003577
Segment 504 score:	0.4285054209595814
Segment 505 score:	0.4558041025655225
Segment 506 score:	0.41801538335590926
Segment 507 score:	0.2966931158345862
Segment 508 score:	0.3267033825089077
Segment 509 score:	0.5186432362434497
Segme

Segment 700 score:	0.2922359613940338
Segment 701 score:	0.39814257848729623
Segment 702 score:	0.38323146921766704
Segment 703 score:	0.4388894695868488
Segment 704 score:	0.48056303603136363
Segment 705 score:	0.39045285316630113
Segment 706 score:	0.43498493234445484
Segment 707 score:	0.5032445255162975
Segment 708 score:	0.4165080509629183
Segment 709 score:	0.4050209599555207
Segment 710 score:	0.45564589019031254
Segment 711 score:	0.42360929193064123
Segment 712 score:	0.46327221355883524
Segment 713 score:	0.48174664797367317
Segment 714 score:	0.393217091824872
Segment 715 score:	0.40652737193389404
Segment 716 score:	0.48835266897191426
Segment 717 score:	0.4636776986678002
Segment 718 score:	0.42358671533254516
Segment 719 score:	0.3296049128771869
Segment 720 score:	0.46016431954426623
Segment 721 score:	0.36413048965769773
Segment 722 score:	0.41223696193356885
Segment 723 score:	0.39531282374261417
Segment 724 score:	0.451724939360608
Segment 725 score:	0.445053820309396

Segment 916 score:	0.3272244992625853
Segment 917 score:	0.2879138186004863
Segment 918 score:	0.3461072889435516
Segment 919 score:	0.41492088896268314
Segment 920 score:	0.43738606639031613
Segment 921 score:	0.37196285988251204
Segment 922 score:	0.3813770488353271
Segment 923 score:	0.33776679042508045
Segment 924 score:	0.3436422128652977
Segment 925 score:	0.2916427453277583
Segment 926 score:	0.3359344342113794
Segment 927 score:	0.40684146513597697
Segment 928 score:	0.3565024918702805
Segment 929 score:	0.44536155368456243
Segment 930 score:	0.3329079946988472
Segment 931 score:	0.4548241726460594
Segment 932 score:	0.43383292243106664
Segment 933 score:	0.48798506179044154
Segment 934 score:	0.37291485974180694
Segment 935 score:	0.3240991206975447
Segment 936 score:	0.3922737975276112
Segment 937 score:	0.37435109878631284
Segment 938 score:	0.4240437643060321
Segment 939 score:	0.23462291546275188
Segment 940 score:	0.42849888630623284
Segment 941 score:	0.28022470949617445

Segment 1161 score:	0.25901144738342213
Segment 1162 score:	0.3130823139025152
Segment 1163 score:	0.5075977857802834
Segment 1164 score:	0.9156661786237191
Segment 1165 score:	0.4026225087776292
Segment 1166 score:	0.40720722059392866
Segment 1167 score:	0.4238622181436537
Segment 1168 score:	1.0
Segment 1169 score:	1.0
Segment 1170 score:	0.3473961178344186
Segment 1171 score:	0.3250375569817789
Segment 1172 score:	0.3609933659768943
Segment 1173 score:	0.2897120126693641
Segment 1174 score:	0.23175816363171575
Segment 1175 score:	0.3203599380256274
Segment 1176 score:	0.3417083216658607
Segment 1177 score:	0.3055821429631054
Segment 1178 score:	0.2976143566354979
Segment 1179 score:	0.3571251469661987
Segment 1180 score:	0.2796024099369055
Segment 1181 score:	0.39015359220759044
Segment 1182 score:	0.23392060419707353
Segment 1183 score:	0.3667277580752048
Segment 1184 score:	0.5012433646993247
Segment 1185 score:	0.34747637996441444
Segment 1186 score:	0.24966620095341607
Segment 1

Segment 1410 score:	0.47599456944850355
Segment 1411 score:	0.5097176584122908
Segment 1412 score:	0.348764627473456
Segment 1413 score:	0.26896508698071425
Segment 1414 score:	0.2692295197876298
Segment 1415 score:	0.4322168811987067
Segment 1416 score:	0.3513357779280771
Segment 1417 score:	0.29325471220340815
Segment 1418 score:	0.3734778343666339
Segment 1419 score:	0.3595611223729307
Segment 1420 score:	0.5298138617701306
Segment 1421 score:	0.37380052647370415
Segment 1422 score:	0.40349691766777607
Segment 1423 score:	0.39995607244024933
Segment 1424 score:	0.2705385696409631
Segment 1425 score:	0.30614710637840425
Segment 1426 score:	0.22687600542405162
Segment 1427 score:	0.20049880680545504
Segment 1428 score:	0.2763745681508991
Segment 1429 score:	0.2742600930271725
Segment 1430 score:	0.23142209212285264
Segment 1431 score:	0.3712491725744933
Segment 1432 score:	0.2865565112199152
Segment 1433 score:	0.37261244231221147
Segment 1434 score:	0.2528514745769644
Segment 1435 sc

Segment 1636 score:	0.28961401628810535
Segment 1637 score:	0.35022130340503915
Segment 1638 score:	0.2843569771414218
Segment 1639 score:	0.2551201893691883
Segment 1640 score:	0.25841183429693404
Segment 1641 score:	0.31796890060355193
Segment 1642 score:	0.33758825212528437
Segment 1643 score:	0.3080475875642672
Segment 1644 score:	0.32333051838964905
Segment 1645 score:	0.3391384548163475
Segment 1646 score:	0.3830084487704514
Segment 1647 score:	0.3309323495476561
Segment 1648 score:	0.36252993278224804
Segment 1649 score:	0.3681538900704413
Segment 1650 score:	0.36767493097102677
Segment 1651 score:	0.3271970662369731
Segment 1652 score:	0.3066214321581514
Segment 1653 score:	0.3667103049778499
Segment 1654 score:	0.3775029906686263
Segment 1655 score:	0.3517465844088124
Segment 1656 score:	0.31953380256624603
Segment 1657 score:	0.27634424010582215
Segment 1658 score:	0.38125828651399474
Segment 1659 score:	0.3231306426166374
Segment 1660 score:	0.3074762685740739
Segment 1661 s

Segment 1849 score:	0.32025579612860466
Segment 1850 score:	0.31660498145372534
Segment 1851 score:	0.2739576002775102
Segment 1852 score:	0.34392802799418143
Segment 1853 score:	0.26060628396228125
Segment 1854 score:	0.2527089382152297
Segment 1855 score:	0.3470083673453662
Segment 1856 score:	0.30065529240704775
Segment 1857 score:	0.23327007925700702
Segment 1858 score:	0.24113550594057678
Segment 1859 score:	0.1750814641209297
Segment 1860 score:	0.21292714422560827
Segment 1861 score:	0.2497232674456085
Segment 1862 score:	0.31723690938067034

System level statistics:


           Test Matches                  Reference Matches
Stage      Content  Function    Total    Content  Function    Total
1            16805     12492    29297      16805     12492    29297
2              107         1      108        107         1      108
3               93       130      223        102       121      223
4              463       818     1281        519       954     1473
Total        17468

## Tilb-NMT

<a href="../data/webnlg2017/submissions/tilburg/report.pdf">report</a>

In [18]:
!java -Xmx2G -jar ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-1_all-cat.txt ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.meteor -l en -norm -r 3 -a ../evaluation/webnlg2017/meteor-1.5/data/paraphrase-en.gz

Meteor version: 1.5

Eval ID:        meteor-1.5-wo-en-norm-0.85_0.2_0.6_0.75-ex_st_sy_pa-1.0_0.6_0.8_0.6

Language:       English
Format:         plaintext
Task:           Ranking
Modules:        exact stem synonym paraphrase
Weights:        1.0 0.6 0.8 0.6
Parameters:     0.85 0.2 0.6 0.75

Segment 1 score:	0.4784139038247433
Segment 2 score:	0.6301425531322936
Segment 3 score:	1.0
Segment 4 score:	0.39922038109462604
Segment 5 score:	1.0
Segment 6 score:	1.0
Segment 7 score:	1.0
Segment 8 score:	1.0
Segment 9 score:	0.5022008728792771
Segment 10 score:	0.45959871966352694
Segment 11 score:	0.29337191840741256
Segment 12 score:	1.0
Segment 13 score:	1.0
Segment 14 score:	0.5183550629438616
Segment 15 score:	0.6049275687106835
Segment 16 score:	0.5281694855074338
Segment 17 score:	1.0
Segment 18 score:	0.534247105415274
Segment 19 score:	0.4133681591895461
Segment 20 score:	0.5759492774652312
Segment 21 score:	0.4433994428440363
Segment 22 score:	1.0
Segment 23 score:	0.571687311571699

Segment 234 score:	0.3110430676901677
Segment 235 score:	0.23726436599331863
Segment 236 score:	0.3610745127532758
Segment 237 score:	0.3502762395001597
Segment 238 score:	0.46105400914532985
Segment 239 score:	0.540658259752086
Segment 240 score:	0.4884196641720276
Segment 241 score:	0.2786356296582109
Segment 242 score:	0.5517528317835068
Segment 243 score:	0.4620231764089805
Segment 244 score:	0.28669671792799706
Segment 245 score:	0.4661067329412456
Segment 246 score:	0.3716061410145492
Segment 247 score:	0.45321494543563445
Segment 248 score:	0.36334260178948796
Segment 249 score:	0.27761446512796883
Segment 250 score:	0.3038197416516659
Segment 251 score:	0.35055411802484165
Segment 252 score:	0.34200584081350927
Segment 253 score:	0.46205832276351866
Segment 254 score:	0.2921317199536426
Segment 255 score:	0.41539390001948
Segment 256 score:	0.3696834996955311
Segment 257 score:	0.4267024276129955
Segment 258 score:	0.3440787890925326
Segment 259 score:	0.3093536758413575
Segmen

Segment 454 score:	0.39960696187699407
Segment 455 score:	0.41859690741033556
Segment 456 score:	0.3474960515792824
Segment 457 score:	0.4126116523333705
Segment 458 score:	0.46491277941433906
Segment 459 score:	0.5238659993762276
Segment 460 score:	0.3738156558646523
Segment 461 score:	0.3591349599558423
Segment 462 score:	0.4512997777691759
Segment 463 score:	0.31523227006963395
Segment 464 score:	0.3899344179774085
Segment 465 score:	0.4137526124230617
Segment 466 score:	0.36230433446803134
Segment 467 score:	0.38213572961786046
Segment 468 score:	0.46309812489658747
Segment 469 score:	0.45203491103824994
Segment 470 score:	0.3165532127027191
Segment 471 score:	0.3063838532691099
Segment 472 score:	0.4665195566494755
Segment 473 score:	0.42631469828871693
Segment 474 score:	0.42593677360479876
Segment 475 score:	0.483674377458173
Segment 476 score:	0.40248420235632426
Segment 477 score:	0.3075026930050725
Segment 478 score:	0.4546779037026651
Segment 479 score:	0.4733767208468752
Se

Segment 689 score:	0.4423661504258912
Segment 690 score:	0.4627168638154629
Segment 691 score:	0.33356647683289
Segment 692 score:	0.352664607480958
Segment 693 score:	0.35191473103870186
Segment 694 score:	0.4177976952843948
Segment 695 score:	0.3147326049153161
Segment 696 score:	0.39792902152265036
Segment 697 score:	0.3826115378410759
Segment 698 score:	0.34518167590938154
Segment 699 score:	0.3186921098407438
Segment 700 score:	0.42050765257531125
Segment 701 score:	0.24224424704919628
Segment 702 score:	0.38330649630076696
Segment 703 score:	1.0
Segment 704 score:	0.40864425892258677
Segment 705 score:	0.3982154946548218
Segment 706 score:	0.4159444366177557
Segment 707 score:	0.41360289133592754
Segment 708 score:	0.45596652634885876
Segment 709 score:	0.3490024220795219
Segment 710 score:	0.4339745176162019
Segment 711 score:	0.4271678084702641
Segment 712 score:	0.4095088830278493
Segment 713 score:	0.590792561565652
Segment 714 score:	0.42658768165547234
Segment 715 score:	0.

Segment 918 score:	0.35294958086311345
Segment 919 score:	0.2985627744008704
Segment 920 score:	0.4490582600386075
Segment 921 score:	0.3924649023329252
Segment 922 score:	0.3876219595689237
Segment 923 score:	0.3815173546947517
Segment 924 score:	0.3566761408942985
Segment 925 score:	0.36391200812874314
Segment 926 score:	0.21151600163580028
Segment 927 score:	0.4201873681767674
Segment 928 score:	0.363165838023336
Segment 929 score:	0.4488386648760259
Segment 930 score:	0.41894360510510037
Segment 931 score:	0.34574721773043393
Segment 932 score:	0.33123757205566845
Segment 933 score:	0.225001869037084
Segment 934 score:	0.3026856416660649
Segment 935 score:	0.3347930786695909
Segment 936 score:	0.36404199415079314
Segment 937 score:	0.2783694481114217
Segment 938 score:	0.3085462422567362
Segment 939 score:	0.1928726211024471
Segment 940 score:	0.3215350284131507
Segment 941 score:	0.22622003473676316
Segment 942 score:	0.2908348750697151
Segment 943 score:	0.32547982946467635
Segme

Segment 1194 score:	0.24441360522727398
Segment 1195 score:	0.34934616196902296
Segment 1196 score:	0.44706598736762876
Segment 1197 score:	0.47249068406310357
Segment 1198 score:	0.2485764758828109
Segment 1199 score:	0.47249068406310357
Segment 1200 score:	0.3784400508804504
Segment 1201 score:	0.4250123326878432
Segment 1202 score:	0.41467370600357245
Segment 1203 score:	1.0
Segment 1204 score:	0.38589395532059473
Segment 1205 score:	0.3894893071763433
Segment 1206 score:	0.349563238302421
Segment 1207 score:	0.47476801591789863
Segment 1208 score:	0.447752250844953
Segment 1209 score:	0.4062840465791614
Segment 1210 score:	0.40595014191547496
Segment 1211 score:	0.22065502547154875
Segment 1212 score:	0.22345264138680326
Segment 1213 score:	0.36035865954465807
Segment 1214 score:	0.3936555294319767
Segment 1215 score:	0.4608084914356292
Segment 1216 score:	0.36901518954514423
Segment 1217 score:	0.2797479719018012
Segment 1218 score:	0.39638818670924064
Segment 1219 score:	0.471229

Segment 1404 score:	0.3690343663326745
Segment 1405 score:	0.4265764857143998
Segment 1406 score:	0.4182436204182491
Segment 1407 score:	0.34988608798371706
Segment 1408 score:	0.2843612398259687
Segment 1409 score:	0.4605979908006276
Segment 1410 score:	0.39944845001859325
Segment 1411 score:	0.41620462517869183
Segment 1412 score:	0.37059977220949303
Segment 1413 score:	0.2199296649313854
Segment 1414 score:	0.3029960703926709
Segment 1415 score:	0.2944585614035061
Segment 1416 score:	0.34392577368004934
Segment 1417 score:	0.335559884348435
Segment 1418 score:	0.34480958005829426
Segment 1419 score:	0.3266308616956521
Segment 1420 score:	0.26775929997187076
Segment 1421 score:	0.3429873980066845
Segment 1422 score:	0.2724464234915794
Segment 1423 score:	0.2748522099749015
Segment 1424 score:	0.25146662691271066
Segment 1425 score:	0.27340770196681446
Segment 1426 score:	0.32060352407099135
Segment 1427 score:	0.16709515379224849
Segment 1428 score:	0.17474352664699316
Segment 1429 s

Segment 1613 score:	0.2264488894595099
Segment 1614 score:	0.173791429968101
Segment 1615 score:	0.18064135663749475
Segment 1616 score:	0.34900409264821863
Segment 1617 score:	0.33363386456389993
Segment 1618 score:	0.30106779483491763
Segment 1619 score:	0.3874521148670191
Segment 1620 score:	0.30036859127282667
Segment 1621 score:	0.35542602046421146
Segment 1622 score:	0.37297901166758535
Segment 1623 score:	0.2793942291069477
Segment 1624 score:	0.3406025652023053
Segment 1625 score:	0.3802907942066456
Segment 1626 score:	0.3348842518082644
Segment 1627 score:	0.3436059284008939
Segment 1628 score:	0.2604903868759761
Segment 1629 score:	0.3022428765999281
Segment 1630 score:	0.3472693309300994
Segment 1631 score:	0.28319034918620073
Segment 1632 score:	0.2723729793686601
Segment 1633 score:	0.16411254831274028
Segment 1634 score:	0.34646286554063427
Segment 1635 score:	0.34672378276286014
Segment 1636 score:	0.32052653502492884
Segment 1637 score:	0.20907121985171695
Segment 1638 

Segment 1846 score:	0.33766797999064607
Segment 1847 score:	0.25933049810705006
Segment 1848 score:	0.40588923647672115
Segment 1849 score:	0.22912957105183074
Segment 1850 score:	0.2227595838638664
Segment 1851 score:	0.3059137836510195
Segment 1852 score:	0.26401501767338476
Segment 1853 score:	0.33952348783956
Segment 1854 score:	0.2900509920228894
Segment 1855 score:	0.22184368764020707
Segment 1856 score:	0.2698005926515317
Segment 1857 score:	0.2872652018721752
Segment 1858 score:	0.27181920013413313
Segment 1859 score:	0.2552484280124004
Segment 1860 score:	0.23869414089148128
Segment 1861 score:	0.2792909459803795
Segment 1862 score:	0.2629358267379688

System level statistics:


           Test Matches                  Reference Matches
Stage      Content  Function    Total    Content  Function    Total
1            16270     11757    28027      16270     11757    28027
2              102         1      103        102         1      103
3              156       180      336   

## Adapt

<a href="../data/webnlg2017/submissions/adaptCenter/WebNLG_system.pdf">report</a>

In [19]:
!java -Xmx2G -jar ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/ADAPT_Centre_all-cat.txt ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.meteor -l en -norm -r 3 -a ../evaluation/webnlg2017/meteor-1.5/data/paraphrase-en.gz

Meteor version: 1.5

Eval ID:        meteor-1.5-wo-en-norm-0.85_0.2_0.6_0.75-ex_st_sy_pa-1.0_0.6_0.8_0.6

Language:       English
Format:         plaintext
Task:           Ranking
Modules:        exact stem synonym paraphrase
Weights:        1.0 0.6 0.8 0.6
Parameters:     0.85 0.2 0.6 0.75

Segment 1 score:	0.5425269137216473
Segment 2 score:	1.0
Segment 3 score:	1.0
Segment 4 score:	0.45331431360109564
Segment 5 score:	1.0
Segment 6 score:	1.0
Segment 7 score:	1.0
Segment 8 score:	1.0
Segment 9 score:	1.0
Segment 10 score:	0.46778818333352057
Segment 11 score:	0.4641985638303238
Segment 12 score:	1.0
Segment 13 score:	1.0
Segment 14 score:	1.0
Segment 15 score:	1.0
Segment 16 score:	0.5338212580268122
Segment 17 score:	0.5523848588957876
Segment 18 score:	1.0
Segment 19 score:	1.0
Segment 20 score:	0.5759492774652312
Segment 21 score:	0.47688665790835266
Segment 22 score:	1.0
Segment 23 score:	0.5647037880749295
Segment 24 score:	0.49885550844723425
Segment 25 score:	0.38776838618822

Segment 262 score:	0.3470980796116069
Segment 263 score:	0.4342848088394785
Segment 264 score:	0.2957480769724148
Segment 265 score:	0.4534283649859343
Segment 266 score:	0.5478818674294312
Segment 267 score:	0.24055564498474036
Segment 268 score:	0.40509430816700565
Segment 269 score:	0.44282222818074596
Segment 270 score:	0.4787358393502326
Segment 271 score:	0.36465342122647554
Segment 272 score:	0.4719072095218859
Segment 273 score:	0.48221789649126934
Segment 274 score:	0.41987287875833573
Segment 275 score:	0.28612389894801477
Segment 276 score:	0.49450869598653385
Segment 277 score:	0.3902002361131234
Segment 278 score:	0.24797299361467706
Segment 279 score:	0.539908461669345
Segment 280 score:	0.9515151515151515
Segment 281 score:	0.32202449125100596
Segment 282 score:	0.29517446445019574
Segment 283 score:	0.3705428331014127
Segment 284 score:	0.5447966751477678
Segment 285 score:	0.42011548586733233
Segment 286 score:	0.5785898173407886
Segment 287 score:	0.5758551163899349
S

Segment 493 score:	0.3604011651616974
Segment 494 score:	0.41390381437889406
Segment 495 score:	0.5429844947195182
Segment 496 score:	0.45356064261118145
Segment 497 score:	0.4856660049203748
Segment 498 score:	0.47649648933866806
Segment 499 score:	0.42413272275625613
Segment 500 score:	0.27225955290839926
Segment 501 score:	0.45264096977747265
Segment 502 score:	0.37889697611861717
Segment 503 score:	0.5484918373862878
Segment 504 score:	0.38308187841695707
Segment 505 score:	0.3357158404847222
Segment 506 score:	0.38379014226213204
Segment 507 score:	0.482732895442215
Segment 508 score:	0.39314763126261554
Segment 509 score:	0.48027516773957457
Segment 510 score:	0.35855870409321877
Segment 511 score:	0.38109235901896615
Segment 512 score:	0.44907756619093664
Segment 513 score:	0.40578368032618006
Segment 514 score:	0.5192421173221803
Segment 515 score:	0.4573721143622308
Segment 516 score:	0.49188577540224054
Segment 517 score:	0.36961574396015134
Segment 518 score:	0.4187893095181

Segment 713 score:	0.5605872755725769
Segment 714 score:	0.5516694013646489
Segment 715 score:	0.528837020916405
Segment 716 score:	0.44624740711351246
Segment 717 score:	0.45861406742655375
Segment 718 score:	0.45222367685847836
Segment 719 score:	0.35533036973125415
Segment 720 score:	0.4312655227761611
Segment 721 score:	0.38230579570387463
Segment 722 score:	0.3760358264127075
Segment 723 score:	0.40724944541867897
Segment 724 score:	0.44683022745114526
Segment 725 score:	0.4911523665587102
Segment 726 score:	0.38067894376596484
Segment 727 score:	0.521798440114199
Segment 728 score:	0.48052577524630663
Segment 729 score:	0.3550128496124028
Segment 730 score:	0.4131831713139914
Segment 731 score:	0.3648273909337807
Segment 732 score:	0.3388463508642791
Segment 733 score:	0.47657929317078457
Segment 734 score:	0.43847749691586096
Segment 735 score:	0.5821621800168812
Segment 736 score:	0.4772135638884167
Segment 737 score:	0.415275739232102
Segment 738 score:	0.5044066172783961
Segm

Segment 932 score:	0.5112309843188269
Segment 933 score:	0.38149138069383365
Segment 934 score:	0.35994432290873213
Segment 935 score:	0.40258248748686676
Segment 936 score:	0.4051470549761586
Segment 937 score:	0.395183121490303
Segment 938 score:	0.42996695543111296
Segment 939 score:	0.3475788118428401
Segment 940 score:	0.4163816013595193
Segment 941 score:	0.2914377154240619
Segment 942 score:	0.31579600148036224
Segment 943 score:	0.42708679838228375
Segment 944 score:	0.5383058678815483
Segment 945 score:	0.5533482030882114
Segment 946 score:	0.45078279868834564
Segment 947 score:	0.5031964641138407
Segment 948 score:	0.5593565218627174
Segment 949 score:	0.5543981043958222
Segment 950 score:	0.43157904970151456
Segment 951 score:	0.36632588570175356
Segment 952 score:	0.4315496672506311
Segment 953 score:	0.4599728825493501
Segment 954 score:	0.5624657990577963
Segment 955 score:	0.5002886421510091
Segment 956 score:	0.39305441631232024
Segment 957 score:	0.3830000756254275
Seg

Segment 1154 score:	0.09775967413441955
Segment 1155 score:	0.23817201344442718
Segment 1156 score:	0.27022057878330696
Segment 1157 score:	0.24152761197523692
Segment 1158 score:	0.11396011396011396
Segment 1159 score:	0.3948219776748272
Segment 1160 score:	0.289654949761442
Segment 1161 score:	0.19849569999505995
Segment 1162 score:	0.23162015392264168
Segment 1163 score:	0.29730776765873823
Segment 1164 score:	0.12030075187969927
Segment 1165 score:	0.09634388738290195
Segment 1166 score:	0.06460807600950119
Segment 1167 score:	0.2275737763763891
Segment 1168 score:	0.41057826767830624
Segment 1169 score:	0.587557873227437
Segment 1170 score:	0.25136340132304463
Segment 1171 score:	0.14782069448440271
Segment 1172 score:	0.07747503527678463
Segment 1173 score:	0.37900634671594496
Segment 1174 score:	0.16135609184894423
Segment 1175 score:	0.18595551247989967
Segment 1176 score:	0.18604651162790697
Segment 1177 score:	0.11506849315068493
Segment 1178 score:	0.1827286656692112
Segment

Segment 1383 score:	0.27683376812639043
Segment 1384 score:	0.09342933471379208
Segment 1385 score:	0.09284610131116773
Segment 1386 score:	0.17000495890057804
Segment 1387 score:	0.1863760263857108
Segment 1388 score:	0.24450687617553218
Segment 1389 score:	0.23277321130810652
Segment 1390 score:	0.18357725666088193
Segment 1391 score:	0.2377518762086175
Segment 1392 score:	0.2680102789914425
Segment 1393 score:	0.22261382067658006
Segment 1394 score:	0.2908657430467824
Segment 1395 score:	0.21734987834112054
Segment 1396 score:	0.13398767550437957
Segment 1397 score:	0.2337242062464961
Segment 1398 score:	0.22781251683316278
Segment 1399 score:	0.17360412007373072
Segment 1400 score:	0.2281442419664108
Segment 1401 score:	0.15310975625658
Segment 1402 score:	0.18153888847202296
Segment 1403 score:	0.1779644983067036
Segment 1404 score:	0.34820653339751517
Segment 1405 score:	0.35378864455578446
Segment 1406 score:	0.33746262330866533
Segment 1407 score:	0.14433032351650102
Segment 14

Segment 1600 score:	0.188347012416498
Segment 1601 score:	0.2075609153396736
Segment 1602 score:	0.2600388916253148
Segment 1603 score:	0.15360667993217894
Segment 1604 score:	0.17962961834615693
Segment 1605 score:	0.20727050649013512
Segment 1606 score:	0.19073310996121381
Segment 1607 score:	0.149754048480915
Segment 1608 score:	0.1465193139372473
Segment 1609 score:	0.1433729102929799
Segment 1610 score:	0.10763169242254236
Segment 1611 score:	0.19122053016524557
Segment 1612 score:	0.15908982022965962
Segment 1613 score:	0.14679679497241516
Segment 1614 score:	0.12329947064577783
Segment 1615 score:	0.1614529714187602
Segment 1616 score:	0.10568191941595165
Segment 1617 score:	0.14231091276128455
Segment 1618 score:	0.25646693262943654
Segment 1619 score:	0.15778429288925608
Segment 1620 score:	0.23885747428207602
Segment 1621 score:	0.2690505360738917
Segment 1622 score:	0.10456200620800431
Segment 1623 score:	0.14699667230046806
Segment 1624 score:	0.15171223916257726
Segment 16

Segment 1814 score:	0.1463194958721933
Segment 1815 score:	0.30140867360067647
Segment 1816 score:	0.23510685153780375
Segment 1817 score:	0.17761873814318377
Segment 1818 score:	0.21044841722636454
Segment 1819 score:	0.2877322433315879
Segment 1820 score:	0.1914263614372043
Segment 1821 score:	0.19562635895782673
Segment 1822 score:	0.13328027926430255
Segment 1823 score:	0.13717252577980063
Segment 1824 score:	0.17069835904356748
Segment 1825 score:	0.13116206015041826
Segment 1826 score:	0.23055064817168897
Segment 1827 score:	0.24783788649812033
Segment 1828 score:	0.20306147868680155
Segment 1829 score:	0.07602151840723784
Segment 1830 score:	0.1537471106989573
Segment 1831 score:	0.20551833329164695
Segment 1832 score:	0.12425762009929582
Segment 1833 score:	0.22823192013692753
Segment 1834 score:	0.18412308795280852
Segment 1835 score:	0.1100480996199237
Segment 1836 score:	0.4666942758437012
Segment 1837 score:	0.29568966665908225
Segment 1838 score:	0.3123682105175042
Segment

## PKUWriter

<a href="../data/webnlg2017/submissions/pkuwriter/PKUWriter_system.pdf">report</a>

In [20]:
!java -Xmx2G -jar ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/PKUWriter_all-cat.txt ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.meteor -l en -norm -r 3 -a ../evaluation/webnlg2017/meteor-1.5/data/paraphrase-en.gz

Meteor version: 1.5

Eval ID:        meteor-1.5-wo-en-norm-0.85_0.2_0.6_0.75-ex_st_sy_pa-1.0_0.6_0.8_0.6

Language:       English
Format:         plaintext
Task:           Ranking
Modules:        exact stem synonym paraphrase
Weights:        1.0 0.6 0.8 0.6
Parameters:     0.85 0.2 0.6 0.75

Segment 1 score:	0.5425269137216473
Segment 2 score:	1.0
Segment 3 score:	0.5254383607629042
Segment 4 score:	0.5183550629438616
Segment 5 score:	1.0
Segment 6 score:	1.0
Segment 7 score:	1.0
Segment 8 score:	1.0
Segment 9 score:	0.5022008728792771
Segment 10 score:	0.520813830585198
Segment 11 score:	0.3879378244241979
Segment 12 score:	0.5318013081707974
Segment 13 score:	0.5523848588957876
Segment 14 score:	1.0
Segment 15 score:	0.6049275687106835
Segment 16 score:	0.4706054944867075
Segment 17 score:	0.5523848588957876
Segment 18 score:	0.534247105415274
Segment 19 score:	1.0
Segment 20 score:	1.0
Segment 21 score:	1.0
Segment 22 score:	0.4959693813968479
Segment 23 score:	0.5647037880749295
Se

Segment 239 score:	0.24681505696738312
Segment 240 score:	0.5145000511076817
Segment 241 score:	0.29995479607466863
Segment 242 score:	0.564422315781808
Segment 243 score:	0.34330935775077226
Segment 244 score:	0.35235040521746175
Segment 245 score:	0.3451302771264373
Segment 246 score:	0.2126871175691804
Segment 247 score:	0.3502042976478264
Segment 248 score:	0.2740614390002918
Segment 249 score:	0.28641983070721216
Segment 250 score:	0.3667880028266254
Segment 251 score:	1.0
Segment 252 score:	0.5643018704905983
Segment 253 score:	0.5643018704905983
Segment 254 score:	0.31749952155264555
Segment 255 score:	0.33715460934755553
Segment 256 score:	0.44019747992143804
Segment 257 score:	1.0
Segment 258 score:	0.39705099501101143
Segment 259 score:	0.21262229982589453
Segment 260 score:	0.1254720677588595
Segment 261 score:	0.38706726349683035
Segment 262 score:	0.3470980796116069
Segment 263 score:	0.4401453933129926
Segment 264 score:	0.290659003899214
Segment 265 score:	0.296476173532

Segment 484 score:	0.2793355069082727
Segment 485 score:	0.28354310013905254
Segment 486 score:	0.2830647078607353
Segment 487 score:	0.2555112158826929
Segment 488 score:	0.1375772016754086
Segment 489 score:	0.2739254374971646
Segment 490 score:	0.2509334059404337
Segment 491 score:	0.22782651676825103
Segment 492 score:	0.2692253787326894
Segment 493 score:	0.3546522409853984
Segment 494 score:	0.13364252658111797
Segment 495 score:	0.3665984008413158
Segment 496 score:	0.24137338642695091
Segment 497 score:	0.37278813821890217
Segment 498 score:	0.2764001381027305
Segment 499 score:	0.2569490835185429
Segment 500 score:	0.19637435487353
Segment 501 score:	0.3428974992213139
Segment 502 score:	0.3040013297492375
Segment 503 score:	0.21320827972772377
Segment 504 score:	0.26545898268139856
Segment 505 score:	0.2740495390155386
Segment 506 score:	0.2537730134718598
Segment 507 score:	0.25876721776912365
Segment 508 score:	0.24351179094378053
Segment 509 score:	0.3641370268194227
Segme

Segment 706 score:	0.5030609692920276
Segment 707 score:	0.478582494379749
Segment 708 score:	0.42539437018351206
Segment 709 score:	0.4477927602852357
Segment 710 score:	0.43202965272367977
Segment 711 score:	0.36506619781135635
Segment 712 score:	0.4085737097156079
Segment 713 score:	0.5207842142820746
Segment 714 score:	0.3715749616169497
Segment 715 score:	0.4601800389754946
Segment 716 score:	0.5222649403740741
Segment 717 score:	0.4591790624346959
Segment 718 score:	0.3256907138218899
Segment 719 score:	0.4091155118599724
Segment 720 score:	0.4487387159819047
Segment 721 score:	0.42349623781352574
Segment 722 score:	0.3714215767506299
Segment 723 score:	0.3299838916987221
Segment 724 score:	0.4883080222876038
Segment 725 score:	0.4911523665587102
Segment 726 score:	0.43541589083003857
Segment 727 score:	0.4094816494676884
Segment 728 score:	0.5428383161134563
Segment 729 score:	0.43368386346254073
Segment 730 score:	0.5203115872620963
Segment 731 score:	0.261422444983325
Segment 

Segment 939 score:	0.3130870340313216
Segment 940 score:	0.30569468159001684
Segment 941 score:	0.2805416599026588
Segment 942 score:	0.3451655894129101
Segment 943 score:	0.41761353289093467
Segment 944 score:	0.49699723090756587
Segment 945 score:	0.35552073153407854
Segment 946 score:	0.33499085263921824
Segment 947 score:	0.42804662928492293
Segment 948 score:	0.35968789513220223
Segment 949 score:	0.3263377823974613
Segment 950 score:	0.5509155438742623
Segment 951 score:	0.33976079137430903
Segment 952 score:	0.2700973138110048
Segment 953 score:	0.269597803192334
Segment 954 score:	0.555190944318092
Segment 955 score:	0.46823367818871964
Segment 956 score:	0.21656626205301774
Segment 957 score:	0.30126656952601877
Segment 958 score:	0.47962405705136374
Segment 959 score:	0.44894969540402857
Segment 960 score:	0.37734659754533983
Segment 961 score:	0.4326453340299844
Segment 962 score:	0.5165985536520424
Segment 963 score:	0.40922876656524926
Segment 964 score:	0.4022452845345087

Segment 1193 score:	0.3559411248455657
Segment 1194 score:	0.41470047692260925
Segment 1195 score:	1.0
Segment 1196 score:	1.0
Segment 1197 score:	1.0
Segment 1198 score:	0.4199647350199303
Segment 1199 score:	1.0
Segment 1200 score:	0.39750579767232835
Segment 1201 score:	0.5293960707300696
Segment 1202 score:	0.2862439922231164
Segment 1203 score:	0.23686517224436643
Segment 1204 score:	0.17372167017692397
Segment 1205 score:	0.16238389998908678
Segment 1206 score:	0.4531935922923504
Segment 1207 score:	0.0714420029112911
Segment 1208 score:	0.18773369124679137
Segment 1209 score:	0.13251733724622367
Segment 1210 score:	0.3979651269922228
Segment 1211 score:	0.35509904374638085
Segment 1212 score:	0.4083072877519969
Segment 1213 score:	0.039535914005804675
Segment 1214 score:	0.35144491073603584
Segment 1215 score:	0.1630271884035241
Segment 1216 score:	0.21097014894753563
Segment 1217 score:	0.13887022094895132
Segment 1218 score:	0.08395116372397353
Segment 1219 score:	0.0812245914

Segment 1427 score:	0.31804005988574574
Segment 1428 score:	0.3829383266491319
Segment 1429 score:	0.332387103076357
Segment 1430 score:	0.3562217188727992
Segment 1431 score:	0.07448987517807354
Segment 1432 score:	0.11476453255557895
Segment 1433 score:	0.03296703296703297
Segment 1434 score:	0.4182045499088988
Segment 1435 score:	0.35304541251486155
Segment 1436 score:	0.11047248517706615
Segment 1437 score:	0.39612538201632425
Segment 1438 score:	0.40441414470775144
Segment 1439 score:	0.30477723427661674
Segment 1440 score:	0.3598555131878579
Segment 1441 score:	0.39031345357179903
Segment 1442 score:	0.2941901495381499
Segment 1443 score:	0.3330934770186788
Segment 1444 score:	0.3653399344807807
Segment 1445 score:	0.3280970757800407
Segment 1446 score:	0.3396793454826272
Segment 1447 score:	0.2531305293246708
Segment 1448 score:	0.3148034918326029
Segment 1449 score:	0.2733432891714033
Segment 1450 score:	0.30367827156159716
Segment 1451 score:	0.3094249456637281
Segment 1452 sc

Segment 1645 score:	0.03712405142563731
Segment 1646 score:	0.38625153434327464
Segment 1647 score:	0.1648045833397459
Segment 1648 score:	0.20642870681600875
Segment 1649 score:	0.2730284840795731
Segment 1650 score:	0.2971613401741525
Segment 1651 score:	0.30471296969266537
Segment 1652 score:	0.31017309529558024
Segment 1653 score:	0.10717955042502109
Segment 1654 score:	0.17149145992743317
Segment 1655 score:	0.17933987965977086
Segment 1656 score:	0.14507810539782098
Segment 1657 score:	0.2831786882397337
Segment 1658 score:	0.41588628042508446
Segment 1659 score:	0.33083035393039806
Segment 1660 score:	0.2913697861644797
Segment 1661 score:	0.3259286937489971
Segment 1662 score:	0.11009928423486975
Segment 1663 score:	0.11009928423486975
Segment 1664 score:	0.025889967637540458
Segment 1665 score:	0.045179297890980075
Segment 1666 score:	0.4462010033922003
Segment 1667 score:	0.06434842199971189
Segment 1668 score:	0.23183950702556938
Segment 1669 score:	0.11037487896846869
Segme

## Tilb-Pipeline

<a href="../data/webnlg2017/submissions/tilburg/report.pdf">report</a>

In [21]:
!java -Xmx2G -jar ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-3_all-cat.txt ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.meteor -l en -norm -r 3 -a ../evaluation/webnlg2017/meteor-1.5/data/paraphrase-en.gz

Meteor version: 1.5

Eval ID:        meteor-1.5-wo-en-norm-0.85_0.2_0.6_0.75-ex_st_sy_pa-1.0_0.6_0.8_0.6

Language:       English
Format:         plaintext
Task:           Ranking
Modules:        exact stem synonym paraphrase
Weights:        1.0 0.6 0.8 0.6
Parameters:     0.85 0.2 0.6 0.75

Segment 1 score:	0.31944770780353793
Segment 2 score:	0.5265114412780284
Segment 3 score:	0.5254383607629042
Segment 4 score:	0.39922038109462604
Segment 5 score:	1.0
Segment 6 score:	0.4227364356652105
Segment 7 score:	0.47470010600213364
Segment 8 score:	0.4238810371140796
Segment 9 score:	0.4199028508389832
Segment 10 score:	0.45959871966352694
Segment 11 score:	0.0
Segment 12 score:	0.4651683609700817
Segment 13 score:	1.0
Segment 14 score:	1.0
Segment 15 score:	0.3798996356223708
Segment 16 score:	0.5338212580268122
Segment 17 score:	1.0
Segment 18 score:	0.3546503291117034
Segment 19 score:	0.23540188682825075
Segment 20 score:	0.4046573359116055
Segment 21 score:	0.4433994428440363
Segment 2

Segment 245 score:	0.2890985574825629
Segment 246 score:	0.35433493888143325
Segment 247 score:	0.2665931637359631
Segment 248 score:	0.39936368397799704
Segment 249 score:	0.23285983091244364
Segment 250 score:	0.30903326932241576
Segment 251 score:	0.4034206669912045
Segment 252 score:	0.2601547600407104
Segment 253 score:	0.300874658000778
Segment 254 score:	0.333481328607147
Segment 255 score:	0.36212509909487867
Segment 256 score:	0.4013043020465287
Segment 257 score:	0.3975159758471615
Segment 258 score:	0.42844964601361235
Segment 259 score:	0.23948076780595529
Segment 260 score:	0.12271539165634568
Segment 261 score:	0.3915171294514038
Segment 262 score:	0.3520957516338639
Segment 263 score:	0.3943605840009644
Segment 264 score:	0.320258627218146
Segment 265 score:	0.40103574318060975
Segment 266 score:	0.22099447513812157
Segment 267 score:	0.4975189671022485
Segment 268 score:	0.320258627218146
Segment 269 score:	0.942857142857143
Segment 270 score:	0.23426852015381447
Segmen

Segment 467 score:	0.37935716462077923
Segment 468 score:	0.342942321451926
Segment 469 score:	0.4543908171456642
Segment 470 score:	0.4780642987744953
Segment 471 score:	0.3883582432320768
Segment 472 score:	0.37966002732063614
Segment 473 score:	0.3451810522584286
Segment 474 score:	0.45354560624356044
Segment 475 score:	0.5010200622064214
Segment 476 score:	0.4386081301832674
Segment 477 score:	0.38438458315302904
Segment 478 score:	0.3691083751990697
Segment 479 score:	0.4234088385432533
Segment 480 score:	0.4779923053794483
Segment 481 score:	0.39121136825951813
Segment 482 score:	0.3364502927402462
Segment 483 score:	0.4160463723207556
Segment 484 score:	0.3405563707037284
Segment 485 score:	0.26924238406450507
Segment 486 score:	0.42053214264166455
Segment 487 score:	0.48155786661306266
Segment 488 score:	0.4069988402718466
Segment 489 score:	0.45825132365349114
Segment 490 score:	0.3439366009531358
Segment 491 score:	0.47611920056691076
Segment 492 score:	0.41015057434785057
Se

Segment 690 score:	0.4564325337222171
Segment 691 score:	0.439125501247195
Segment 692 score:	0.3992697700429584
Segment 693 score:	0.40713600152331325
Segment 694 score:	0.5006826053559766
Segment 695 score:	0.35316207813834194
Segment 696 score:	0.36557204926913645
Segment 697 score:	0.39156755027717494
Segment 698 score:	0.38648287592890734
Segment 699 score:	0.3711848524018217
Segment 700 score:	0.2861430300999694
Segment 701 score:	0.38862904428669964
Segment 702 score:	0.3665163599361275
Segment 703 score:	0.4605583823872216
Segment 704 score:	0.4279739638568042
Segment 705 score:	0.39342582629920925
Segment 706 score:	0.42120052121083745
Segment 707 score:	0.4117679464162127
Segment 708 score:	0.3986373189056646
Segment 709 score:	0.38916079107378476
Segment 710 score:	0.4229763904401501
Segment 711 score:	0.39965925060596824
Segment 712 score:	0.37135285887521485
Segment 713 score:	0.46040800296812273
Segment 714 score:	0.4143289250546103
Segment 715 score:	0.3395703291158986
S

Segment 920 score:	0.4146024265727205
Segment 921 score:	0.4373623408536627
Segment 922 score:	0.40774280408022207
Segment 923 score:	0.423708057271721
Segment 924 score:	0.41050621274855703
Segment 925 score:	0.36947116717571166
Segment 926 score:	0.29999776964916147
Segment 927 score:	0.4072104049049916
Segment 928 score:	0.2969386326840872
Segment 929 score:	0.3654329660532599
Segment 930 score:	0.3707505045109548
Segment 931 score:	0.3742926058734239
Segment 932 score:	0.4259766013475866
Segment 933 score:	0.35661009206920774
Segment 934 score:	0.40995402947116943
Segment 935 score:	0.30843376304620174
Segment 936 score:	0.34105290818530937
Segment 937 score:	0.39316809301786265
Segment 938 score:	0.35368583833280204
Segment 939 score:	0.3057027790978055
Segment 940 score:	0.3539015635449122
Segment 941 score:	0.36485930927155763
Segment 942 score:	0.3302018700818992
Segment 943 score:	0.3883096252061144
Segment 944 score:	0.27654488028673063
Segment 945 score:	0.2676801395437678
S

Segment 1215 score:	0.38983179642590116
Segment 1216 score:	0.4787085940128381
Segment 1217 score:	0.3517198774059452
Segment 1218 score:	0.22583799791071935
Segment 1219 score:	0.4895326953039326
Segment 1220 score:	0.1844793177465533
Segment 1221 score:	0.31060539686214717
Segment 1222 score:	0.22024149644029622
Segment 1223 score:	0.2801919259156008
Segment 1224 score:	0.38239105455490235
Segment 1225 score:	0.47699098168363646
Segment 1226 score:	0.397344486902829
Segment 1227 score:	0.39028263948401143
Segment 1228 score:	0.3832799092226045
Segment 1229 score:	0.3304854282075434
Segment 1230 score:	1.0
Segment 1231 score:	0.25421752643434803
Segment 1232 score:	0.33689175334267635
Segment 1233 score:	0.0
Segment 1234 score:	0.5169608498733645
Segment 1235 score:	0.4661865662450147
Segment 1236 score:	0.0
Segment 1237 score:	0.24614217688751078
Segment 1238 score:	0.30047102325157615
Segment 1239 score:	0.26961406975788954
Segment 1240 score:	0.365141891881862
Segment 1241 score:	0

Segment 1520 score:	0.0
Segment 1521 score:	0.0
Segment 1522 score:	0.36326250521160464
Segment 1523 score:	0.35343264850590145
Segment 1524 score:	0.3834824548065854
Segment 1525 score:	0.0
Segment 1526 score:	0.19628055682172854
Segment 1527 score:	0.2543911925444579
Segment 1528 score:	0.1946361950515418
Segment 1529 score:	0.2982582505985548
Segment 1530 score:	0.3800953141911825
Segment 1531 score:	0.38415111341334535
Segment 1532 score:	0.3036738492374198
Segment 1533 score:	0.344919927977903
Segment 1534 score:	0.38189458315054436
Segment 1535 score:	0.4619677737016387
Segment 1536 score:	0.0
Segment 1537 score:	0.0
Segment 1538 score:	0.3042264477774357
Segment 1539 score:	0.46460224964922325
Segment 1540 score:	0.3814230738965922
Segment 1541 score:	0.311708467886
Segment 1542 score:	0.2667686706036863
Segment 1543 score:	0.3887008295099598
Segment 1544 score:	0.30963778084964116
Segment 1545 score:	0.4560395291866469
Segment 1546 score:	0.436455703462014
Segment 1547 score:	0

Segment 1769 score:	0.33570245986406005
Segment 1770 score:	0.36555218147775287
Segment 1771 score:	0.3364571698796845
Segment 1772 score:	0.2778127507474874
Segment 1773 score:	0.3875736917953203
Segment 1774 score:	0.29696298541976257
Segment 1775 score:	0.3327968178789985
Segment 1776 score:	0.37326684438504615
Segment 1777 score:	0.31654279342324665
Segment 1778 score:	0.39382655822512724
Segment 1779 score:	0.3609198016100228
Segment 1780 score:	0.3776955740571208
Segment 1781 score:	0.0
Segment 1782 score:	0.2505439696252425
Segment 1783 score:	0.28992740808724465
Segment 1784 score:	0.2741557748409711
Segment 1785 score:	0.19481429353067395
Segment 1786 score:	0.2274853737760567
Segment 1787 score:	0.2516384838622495
Segment 1788 score:	0.2369707818875974
Segment 1789 score:	0.0
Segment 1790 score:	0.0
Segment 1791 score:	0.0
Segment 1792 score:	0.26216908841835856
Segment 1793 score:	0.16801218684520053
Segment 1794 score:	0.0
Segment 1795 score:	0.0
Segment 1796 score:	0.34205

## Baseline

In [22]:
!java -Xmx2G -jar ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Baseline_all-cat.txt ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.meteor -l en -norm -r 3 -a ../evaluation/webnlg2017/meteor-1.5/data/paraphrase-en.gz

Meteor version: 1.5

Eval ID:        meteor-1.5-wo-en-norm-0.85_0.2_0.6_0.75-ex_st_sy_pa-1.0_0.6_0.8_0.6

Language:       English
Format:         plaintext
Task:           Ranking
Modules:        exact stem synonym paraphrase
Weights:        1.0 0.6 0.8 0.6
Parameters:     0.85 0.2 0.6 0.75

Segment 1 score:	0.5425269137216473
Segment 2 score:	1.0
Segment 3 score:	1.0
Segment 4 score:	0.5183550629438616
Segment 5 score:	1.0
Segment 6 score:	1.0
Segment 7 score:	1.0
Segment 8 score:	1.0
Segment 9 score:	0.5022008728792771
Segment 10 score:	0.36864071439067725
Segment 11 score:	0.3666547512324887
Segment 12 score:	0.5472108090547545
Segment 13 score:	0.5523848588957876
Segment 14 score:	1.0
Segment 15 score:	0.6049275687106835
Segment 16 score:	0.4706054944867075
Segment 17 score:	0.5523848588957876
Segment 18 score:	1.0
Segment 19 score:	1.0
Segment 20 score:	1.0
Segment 21 score:	1.0
Segment 22 score:	1.0
Segment 23 score:	0.5647037880749295
Segment 24 score:	0.49885550844723425
Segmen

Segment 244 score:	0.3577014874846939
Segment 245 score:	0.36690879579983876
Segment 246 score:	0.27369539250302055
Segment 247 score:	0.37918366650544866
Segment 248 score:	0.2740614390002918
Segment 249 score:	0.28773041070549366
Segment 250 score:	0.3300037736565049
Segment 251 score:	0.5245320422616659
Segment 252 score:	0.39478730053791433
Segment 253 score:	0.4681257397406342
Segment 254 score:	0.18747479828108435
Segment 255 score:	0.4291070682322423
Segment 256 score:	0.14782028491230956
Segment 257 score:	0.3696208475781189
Segment 258 score:	0.6369565473907569
Segment 259 score:	0.28757812678441014
Segment 260 score:	0.08058108677122401
Segment 261 score:	0.47262299353340265
Segment 262 score:	0.3511262971699621
Segment 263 score:	0.3787001073134511
Segment 264 score:	0.29372934548969865
Segment 265 score:	0.3342614116076853
Segment 266 score:	1.0
Segment 267 score:	0.46701032137474074
Segment 268 score:	0.19321818976012364
Segment 269 score:	0.5643018704905983
Segment 270 sc

Segment 482 score:	0.25715457295779975
Segment 483 score:	0.27302872730270955
Segment 484 score:	0.30977180507117325
Segment 485 score:	0.2660331255286175
Segment 486 score:	0.34999693378312696
Segment 487 score:	0.33836413900233936
Segment 488 score:	0.1415426281953055
Segment 489 score:	0.25320226034014054
Segment 490 score:	0.43064483475057463
Segment 491 score:	0.1265470830679988
Segment 492 score:	0.3054959381653361
Segment 493 score:	0.34394905425991656
Segment 494 score:	0.2192336467666324
Segment 495 score:	0.28895763201875285
Segment 496 score:	0.38513815249165556
Segment 497 score:	0.3790530624818
Segment 498 score:	0.2600861118794208
Segment 499 score:	0.2964037632478771
Segment 500 score:	0.24762892026880862
Segment 501 score:	0.258352917461762
Segment 502 score:	0.36306509483354993
Segment 503 score:	0.31195101516227036
Segment 504 score:	0.4257405759285536
Segment 505 score:	0.3748770371723487
Segment 506 score:	0.25097701925507887
Segment 507 score:	0.4466332830965727
Se

Segment 712 score:	0.3736538888629869
Segment 713 score:	0.5046145996290193
Segment 714 score:	0.5292834455537065
Segment 715 score:	0.5159768275138609
Segment 716 score:	0.4374209903748904
Segment 717 score:	0.43273856682774114
Segment 718 score:	0.36926891456973254
Segment 719 score:	0.535100003851662
Segment 720 score:	0.44087810648069087
Segment 721 score:	0.42222255170142625
Segment 722 score:	0.37513189039023254
Segment 723 score:	0.4419208673590823
Segment 724 score:	0.4494848823618047
Segment 725 score:	0.2228335213828589
Segment 726 score:	0.2919198984615355
Segment 727 score:	0.4510881262606572
Segment 728 score:	0.4426577378333252
Segment 729 score:	0.23915662263188436
Segment 730 score:	0.35742979253712664
Segment 731 score:	0.31089008511283556
Segment 732 score:	0.2658617190173955
Segment 733 score:	0.41922980927676434
Segment 734 score:	0.42363692951323995
Segment 735 score:	0.23893447203558277
Segment 736 score:	0.4747817282895916
Segment 737 score:	0.30525381727118583
S

Segment 941 score:	0.3092973561984837
Segment 942 score:	0.2534373838347674
Segment 943 score:	0.40330306535087357
Segment 944 score:	0.5637476508790347
Segment 945 score:	0.40330203767327044
Segment 946 score:	0.3099032228524668
Segment 947 score:	0.3087761437853095
Segment 948 score:	0.3464812296384663
Segment 949 score:	0.36211496929546705
Segment 950 score:	0.45666011904461706
Segment 951 score:	0.3123044305783954
Segment 952 score:	0.40917184853443295
Segment 953 score:	0.27699822041336436
Segment 954 score:	0.41157726587575
Segment 955 score:	0.44294790433723524
Segment 956 score:	0.3871752684177512
Segment 957 score:	0.38390436496417096
Segment 958 score:	0.4011855869475189
Segment 959 score:	0.4592708668703713
Segment 960 score:	0.43750341471079346
Segment 961 score:	0.46119043073459265
Segment 962 score:	0.38755789204846636
Segment 963 score:	0.3934095508777785
Segment 964 score:	0.32342973651632934
Segment 965 score:	0.40189676797089324
Segment 966 score:	0.3521991361847378
S

Segment 1272 score:	0.09860611791439351
Segment 1273 score:	0.39158126539730226
Segment 1274 score:	0.15800537944745305
Segment 1275 score:	0.24661902358103602
Segment 1276 score:	0.25463055723558115
Segment 1277 score:	0.11129085048947143
Segment 1278 score:	0.05600113362618675
Segment 1279 score:	0.05899705014749264
Segment 1280 score:	0.059416445623342175
Segment 1281 score:	0.1959356561607532
Segment 1282 score:	0.053025842603640204
Segment 1283 score:	0.04216470588235294
Segment 1284 score:	0.06546644844517185
Segment 1285 score:	0.209012189328533
Segment 1286 score:	0.15130023640661938
Segment 1287 score:	0.0366412213740458
Segment 1288 score:	0.040336134453781515
Segment 1289 score:	0.08333333333333334
Segment 1290 score:	0.028070175438596492
Segment 1291 score:	0.03855421686746988
Segment 1292 score:	0.04477611940298508
Segment 1293 score:	0.05157815021043494
Segment 1294 score:	0.05015673981191224
Segment 1295 score:	0.025764895330112725
Segment 1296 score:	0.04095063985374771

Segment 1488 score:	0.03864734299516909
Segment 1489 score:	0.19139369922576643
Segment 1490 score:	0.04306300722402085
Segment 1491 score:	0.04283762701064033
Segment 1492 score:	0.025889967637540458
Segment 1493 score:	0.18795809990394843
Segment 1494 score:	0.03243243243243243
Segment 1495 score:	0.06435884226025958
Segment 1496 score:	0.07120970963991634
Segment 1497 score:	0.024804361217927503
Segment 1498 score:	0.1360534394666591
Segment 1499 score:	0.1832124366788838
Segment 1500 score:	0.05072016803871967
Segment 1501 score:	0.03550567134337622
Segment 1502 score:	0.04560570071258907
Segment 1503 score:	0.0548641737395802
Segment 1504 score:	0.10493044691617884
Segment 1505 score:	0.1042723180474417
Segment 1506 score:	0.0577337598247662
Segment 1507 score:	0.05234460196292258
Segment 1508 score:	0.032160804020100506
Segment 1509 score:	0.03265306122448979
Segment 1510 score:	0.039093174067899764
Segment 1511 score:	0.0409402978026876
Segment 1512 score:	0.0442600276625173
Seg

Segment 1700 score:	0.0879799061692116
Segment 1701 score:	0.18144159953208794
Segment 1702 score:	0.047276013064907085
Segment 1703 score:	0.05871690447020066
Segment 1704 score:	0.11979311434881522
Segment 1705 score:	0.12849297343889074
Segment 1706 score:	0.054123358688960274
Segment 1707 score:	0.044456759770763174
Segment 1708 score:	0.08565055762081786
Segment 1709 score:	0.053963334237761625
Segment 1710 score:	0.06345222296949532
Segment 1711 score:	0.03265306122448979
Segment 1712 score:	0.06805530199750247
Segment 1713 score:	0.05525963324621432
Segment 1714 score:	0.04845456731233417
Segment 1715 score:	0.04744197651319751
Segment 1716 score:	0.061781146169996255
Segment 1717 score:	0.1888122014766475
Segment 1718 score:	0.2142715831212768
Segment 1719 score:	0.1697475497608302
Segment 1720 score:	0.028639943211670914
Segment 1721 score:	0.0827775643385315
Segment 1722 score:	0.06743173631863598
Segment 1723 score:	0.05456857091201711
Segment 1724 score:	0.11641703857713095

## UIT-VNU

<a href="../data/webnlg2017/submissions/uit-danglt-clnlp/Phong-Nguyen-Tran_Dang-Tuan-Nguyen.docx">report</a>

In [25]:
!java -Xmx2G -jar ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UIT-DANGNT-CLNLP_all-cat.txt ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.meteor -l en -norm -r 3 -a ../evaluation/webnlg2017/meteor-1.5/data/paraphrase-en.gz

Meteor version: 1.5

Eval ID:        meteor-1.5-wo-en-norm-0.85_0.2_0.6_0.75-ex_st_sy_pa-1.0_0.6_0.8_0.6

Language:       English
Format:         plaintext
Task:           Ranking
Modules:        exact stem synonym paraphrase
Weights:        1.0 0.6 0.8 0.6
Parameters:     0.85 0.2 0.6 0.75

Segment 1 score:	0.4894635482157718
Segment 2 score:	0.6216880466054739
Segment 3 score:	0.5603293785486797
Segment 4 score:	0.5776093927047058
Segment 5 score:	0.4004960070384709
Segment 6 score:	0.41280535374088306
Segment 7 score:	0.5957696165355999
Segment 8 score:	0.5309315227410509
Segment 9 score:	0.521324221008186
Segment 10 score:	0.46778818333352057
Segment 11 score:	0.0
Segment 12 score:	0.49479447973501023
Segment 13 score:	0.4443438716486749
Segment 14 score:	0.4907109540390857
Segment 15 score:	0.4978483627462348
Segment 16 score:	0.4706054944867075
Segment 17 score:	0.4443438716486749
Segment 18 score:	0.5212857378611374
Segment 19 score:	0.5038217434220711
Segment 20 score:	0.525779

Segment 251 score:	0.3348718940195518
Segment 252 score:	0.4718719594598886
Segment 253 score:	0.3715856879223934
Segment 254 score:	0.3190966875489524
Segment 255 score:	0.4528315119729106
Segment 256 score:	0.3696834996955311
Segment 257 score:	0.3436425023930487
Segment 258 score:	0.5819478700812771
Segment 259 score:	0.2994722732463825
Segment 260 score:	0.0
Segment 261 score:	0.0
Segment 262 score:	0.3282681657086264
Segment 263 score:	0.3407937135185171
Segment 264 score:	0.33088260649165047
Segment 265 score:	0.0
Segment 266 score:	0.4307521200312841
Segment 267 score:	0.5132377611778421
Segment 268 score:	0.3127146771776743
Segment 269 score:	0.36904169369241957
Segment 270 score:	0.5538026073988654
Segment 271 score:	0.42888702983394555
Segment 272 score:	0.427848623382235
Segment 273 score:	0.34551802626529987
Segment 274 score:	0.5501226457785693
Segment 275 score:	0.0
Segment 276 score:	0.32082384943559955
Segment 277 score:	0.30810082668838146
Segment 278 score:	0.25768617

Segment 541 score:	0.0
Segment 542 score:	0.0
Segment 543 score:	0.33966909185283145
Segment 544 score:	0.40859682666570496
Segment 545 score:	0.0
Segment 546 score:	0.49138866704135836
Segment 547 score:	0.0
Segment 548 score:	0.0
Segment 549 score:	0.3655302593919405
Segment 550 score:	0.3330207034917668
Segment 551 score:	0.0
Segment 552 score:	0.39462601937207303
Segment 553 score:	0.35051360973631246
Segment 554 score:	0.3823782461434606
Segment 555 score:	0.0
Segment 556 score:	0.0
Segment 557 score:	0.0
Segment 558 score:	0.535388401698994
Segment 559 score:	0.4017135054653187
Segment 560 score:	0.34689524471557526
Segment 561 score:	0.45310148081915363
Segment 562 score:	0.0
Segment 563 score:	0.0
Segment 564 score:	0.0
Segment 565 score:	0.34238263817631126
Segment 566 score:	0.28163744217865083
Segment 567 score:	0.0
Segment 568 score:	0.0
Segment 569 score:	0.0
Segment 570 score:	0.0
Segment 571 score:	0.0
Segment 572 score:	0.0
Segment 573 score:	0.29708177330412117
Segment

Segment 887 score:	0.4306001780755499
Segment 888 score:	0.40942713907966166
Segment 889 score:	0.008026755852842809
Segment 890 score:	0.0
Segment 891 score:	0.0
Segment 892 score:	0.4687784205486363
Segment 893 score:	0.0
Segment 894 score:	0.4216847332314868
Segment 895 score:	0.453291860947009
Segment 896 score:	0.0
Segment 897 score:	0.0
Segment 898 score:	0.0
Segment 899 score:	0.0
Segment 900 score:	0.0
Segment 901 score:	0.0
Segment 902 score:	0.0
Segment 903 score:	0.0
Segment 904 score:	0.0
Segment 905 score:	0.012182741116751269
Segment 906 score:	0.010140845070422535
Segment 907 score:	0.0
Segment 908 score:	0.0
Segment 909 score:	0.0
Segment 910 score:	0.0
Segment 911 score:	0.0
Segment 912 score:	0.0
Segment 913 score:	0.0
Segment 914 score:	0.0
Segment 915 score:	0.0
Segment 916 score:	0.0
Segment 917 score:	0.0
Segment 918 score:	0.0
Segment 919 score:	0.0
Segment 920 score:	0.0
Segment 921 score:	0.0
Segment 922 score:	0.0
Segment 923 score:	0.0
Segment 924 score:	0.37

Segment 1211 score:	0.0
Segment 1212 score:	0.0
Segment 1213 score:	0.0
Segment 1214 score:	0.0
Segment 1215 score:	0.0
Segment 1216 score:	0.0
Segment 1217 score:	0.0
Segment 1218 score:	0.0
Segment 1219 score:	0.0
Segment 1220 score:	0.0
Segment 1221 score:	0.0
Segment 1222 score:	0.0
Segment 1223 score:	0.0
Segment 1224 score:	0.0
Segment 1225 score:	0.0
Segment 1226 score:	0.0
Segment 1227 score:	0.0
Segment 1228 score:	0.0
Segment 1229 score:	0.0
Segment 1230 score:	0.5984035034279488
Segment 1231 score:	0.0
Segment 1232 score:	0.0
Segment 1233 score:	0.0
Segment 1234 score:	0.46068191055628765
Segment 1235 score:	0.0
Segment 1236 score:	0.0
Segment 1237 score:	0.0
Segment 1238 score:	0.0
Segment 1239 score:	0.0
Segment 1240 score:	0.0
Segment 1241 score:	0.0
Segment 1242 score:	0.0
Segment 1243 score:	0.0
Segment 1244 score:	0.0
Segment 1245 score:	0.36935886410212576
Segment 1246 score:	0.0
Segment 1247 score:	0.0
Segment 1248 score:	0.0
Segment 1249 score:	0.0
Segment 1250 scor

Segment 1597 score:	0.0
Segment 1598 score:	0.0
Segment 1599 score:	0.0
Segment 1600 score:	0.0
Segment 1601 score:	0.0
Segment 1602 score:	0.0
Segment 1603 score:	0.0
Segment 1604 score:	0.0
Segment 1605 score:	0.0
Segment 1606 score:	0.0
Segment 1607 score:	0.0
Segment 1608 score:	0.0
Segment 1609 score:	0.0
Segment 1610 score:	0.0
Segment 1611 score:	0.0
Segment 1612 score:	0.0
Segment 1613 score:	0.0
Segment 1614 score:	0.0
Segment 1615 score:	0.0
Segment 1616 score:	0.0
Segment 1617 score:	0.0
Segment 1618 score:	0.0
Segment 1619 score:	0.0
Segment 1620 score:	0.0
Segment 1621 score:	0.0
Segment 1622 score:	0.0
Segment 1623 score:	0.0
Segment 1624 score:	0.0
Segment 1625 score:	0.0
Segment 1626 score:	0.0
Segment 1627 score:	0.0
Segment 1628 score:	0.0
Segment 1629 score:	0.0
Segment 1630 score:	0.0
Segment 1631 score:	0.0
Segment 1632 score:	0.0
Segment 1633 score:	0.0
Segment 1634 score:	0.0
Segment 1635 score:	0.0
Segment 1636 score:	0.0
Segment 1637 score:	0.0
Segment 1638 sco

# My models

In [2]:
models = !ls -R -d ../tmp/**/*

models = [m for m in models if m.endswith('_all-cat.txt')]

In [3]:
for model in models:
    
    print(model)
    
    !../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < "$model"
    
    !java -Xmx2G -jar ../evaluation/webnlg2017/meteor-1.5/meteor-1.5.jar "$model" ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference-3ref.meteor -l en -norm -r 3 -a ../evaluation/webnlg2017/meteor-1.5/data/paraphrase-en.gz | tail -1
    
    print()

../tmp/4 - Baseline - Just join triples_c952c4774651e01f72450b3ac8ebcde98ab9f157/4 - Baseline - Just join triples_c952c4774651e01f72450b3ac8ebcde98ab9f157_all-cat.txt
BLEU = 24.83, 64.6/36.0/19.5/11.2 (BP=0.930, ratio=0.932, hyp_len=38801, ref_len=41636)
Final score:            0.33796981192245906

../tmp/5 - Model - Template Based - roots_310dd152c335dfb79d9d63de09095d7f70ce7c7c/5 - Model - Template Based - roots_310dd152c335dfb79d9d63de09095d7f70ce7c7c0_all-cat.txt
BLEU = 36.97, 70.8/45.1/29.9/19.6 (BP=1.000, ratio=1.138, hyp_len=52032, ref_len=45719)
Final score:            0.3920785470630326

../tmp/5 - Model - Template Based - roots_310dd152c335dfb79d9d63de09095d7f70ce7c7c/5 - Model - Template Based - roots_310dd152c335dfb79d9d63de09095d7f70ce7c7c1_all-cat.txt
BLEU = 38.57, 73.4/47.3/31.2/20.5 (BP=1.000, ratio=1.097, hyp_len=49259, ref_len=44910)
Final score:            0.3983897243681605

../tmp/5 - Model - Template Based - roots_310dd152c335dfb79d9d63de09095d7f70ce7c7c/5 - Model